In [1]:
import psycopg2
import keys

def initialize_db(db_name):

    # Edit connection parameters to match your deployment.
    # This connection assumes Postgres has been installed in your local environment or with an appropriately-configured VM
    # PG listens on port 5432 by default
    conn = psycopg2.connect(
        host="localhost",
        user='postgres',
        password=keys.pg_pwd
    )

    conn.autocommit = True
    with conn.cursor() as cur:
        try:
            cur.execute(f"CREATE DATABASE {db_name}")
        except(Exception, psycopg2.DatabaseError) as error:
            print(error)
    conn.close()

    conn = psycopg2.connect( # reconnect to database
        host="localhost",
        dbname=db_name,
        user='postgres',
        password=keys.pg_pwd
    )
    conn.autocommit = True
    with conn.cursor() as cur:
        try:
            # enable pgvector extension
            cur.execute("CREATE EXTENSION vector")

            # initialize tables
            cur.execute("""
                CREATE TABLE linktable (
                    record_id SERIAL PRIMARY KEY,
                    filingname TEXT,
                    filingdate DATE,
                    aenumber VARCHAR(255),
                    attachment BOOLEAN,
                    hyperlink TEXT
                )
            """)
            cur.execute("""
                CREATE TABLE transcripts (
                    transcript_id SERIAL PRIMARY KEY,
                    filingname TEXT,
                    filingdate DATE,
                    aenumber VARCHAR(255),
                    attachment BOOLEAN,
                    hyperlink TEXT,
                    summary TEXT
                )
            """)
            ## adjust embedding dimension to match planned embedding model
            cur.execute("""
                CREATE TABLE pages (
                    page_id SERIAL PRIMARY KEY,
                    embedding vector(1024),
                    page_number INT,
                    parent_doc TEXT,
                    text TEXT
                )
            """)
        except(Exception, psycopg2.DatabaseError) as error:
            print(error)
    conn.close()

initialize_db('caroldb')